In [1]:
import pandas as pd
import numpy as np
import sys
import os
import random
from math import sqrt
sys.path.append('../..')
from modules import utils
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
%matplotlib inline

2023-07-28 21:33:45.934419: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-28 21:33:46.107099: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-28 21:33:46.115926: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-07-28 21:33:46.115942: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudar

In [2]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
os.environ['PYTHONHASHSEED']=str(SEED)
tf.random.set_seed(SEED)

#### The data

In [3]:
kampala_devices = pd.read_csv('../data/kampala_devices.csv', usecols=['lat', 'long', 'id'])
kampala_devices.head()

,id,lat,long
0,930434,0.360209,32.610756
1,718028,0.307500,32.620600
2,912224,0.346460,32.703280
3,930426,0.365500,32.646800
4,930427,0.268900,32.588000


In [4]:
kampala_df = pd.read_csv('../data/kampala_data.csv', parse_dates=['timestamp'])
kampala_df.head()

,site_name,latitude,longitude,city,timestamp,pm2_5_calibrated_value,pm2_5_raw_value,pm10_raw_value,pm10_calibrated_value,site_id,device_number,device_name
0,"Civic Centre, Kampala Central",0.317725,32.592509,Kampala,2021-09-01 00:00:00+00:00,NaN,NaN,NaN,NaN,60d058c8048305120d2d6145,689761,aq_26
1,"Civic Centre, Kampala Central",0.317725,32.592509,Kampala,2021-09-01 01:00:00+00:00,NaN,NaN,NaN,NaN,60d058c8048305120d2d6145,689761,aq_26
2,"Civic Centre, Kampala Central",0.317725,32.592509,Kampala,2021-09-01 02:00:00+00:00,NaN,NaN,NaN,NaN,60d058c8048305120d2d6145,689761,aq_26
3,"Civic Centre, Kampala Central",0.317725,32.592509,Kampala,2021-09-01 03:00:00+00:00,NaN,NaN,NaN,NaN,60d058c8048305120d2d6145,689761,aq_26
4,"Civic Centre, Kampala Central",0.317725,32.592509,Kampala,2021-09-01 04:00:00+00:00,NaN,NaN,NaN,NaN,60d058c8048305120d2d6145,689761,aq_26


In [5]:
final_df = pd.DataFrame()
# device_ids = []
cols = ['timestamp', 'latitude', 'longitude', 'pm2_5_calibrated_value', 'device_number']
# for i, device_id in enumerate(device_ids):
for i, device_id in kampala_devices.id.iteritems():
    device_df = utils.get_device_data(kampala_df, device_id, cols)
#     if len(device_df)!=0:
#         device_ids.append(device_id)
    processed_df = utils.preprocessing(device_df)
#     print(f'{device_id}: {len(processed_df)}')
    final_df = pd.concat([final_df, processed_df])
final_df.reset_index(drop=True, inplace=True)
final_df.head()

,time,latitude,longitude,pm2_5,device_number
0,453031.0,0.356989,32.613888,10.5477,930434
1,453032.0,0.356989,32.613888,16.4250,930434
2,453033.0,0.356989,32.613888,17.7239,930434
3,453034.0,0.356989,32.613888,16.1533,930434
4,453035.0,0.356989,32.613888,18.0123,930434


In [6]:
latitudes = final_df['latitude'].unique()
longitudes = final_df['longitude'].unique()
device_ids = final_df['device_number'].unique()
len(latitudes), len(longitudes), len(device_ids)

(35, 35, 34)

In [7]:
final_df = final_df.drop(['device_number'], axis=1)
final_df.head()

,time,latitude,longitude,pm2_5
0,453031.0,0.356989,32.613888,10.5477
1,453032.0,0.356989,32.613888,16.4250
2,453033.0,0.356989,32.613888,17.7239
3,453034.0,0.356989,32.613888,16.1533
4,453035.0,0.356989,32.613888,18.0123


#### Model training and validation

In [8]:
def ffnn(X_train, y_train, epochs=1000, optimizer='RMSProp', dropout=0.2):
    model = tf.keras.Sequential() 
    model.add(Input(shape=(X_train.shape[1],), name='Input-Layer')) 
#     model.add(Dropout(rate=dropout))
    model.add(Dense(128, activation='relu', name='Hidden-Layer1'))
#     model.add(Dropout(rate=dropout))
    model.add(Dense(32, activation='relu', name='Hidden-Layer2'))
#     model.add(Dropout(rate=dropout))
    model.add(Dense(1, activation='linear', name='Output-Layer')) 

    model.compile(optimizer=optimizer, # default='rmsprop', an algorithm to be used in backpropagation
                  loss=tf.keras.losses.MeanSquaredError(),
                 )
    model.fit(X_train, 
          y_train,
          batch_size=32,
          epochs=epochs, # default=1, Number of epochs to train the model
          callbacks=None,
          validation_split=0.2, 
         )
    return model

In [9]:
# def scale_data(X):
#     scaler = MinMaxScaler()
#     X_scaled = X.copy()
#     X_scaled[:, 0] = scaler.fit_transform(X[:, 0].reshape(-1, 1)).flatten()
#     return X_scaled

#### delete from here

In [10]:
# idx = 0
# device_indices = final_df[final_df.latitude==latitudes[idx]].index
# device_df = jinja_df[jinja_df.device_number == device_ids[idx]]
# assert(len(device_indices) == len(device_df)-device_df.pm2_5_calibrated_value.isna().sum())
# test_df = final_df.loc[device_indices]
# test_df.head()

In [11]:
# train_df = pd.concat([final_df, test_df]).drop_duplicates(keep=False)
# assert(len(train_df.longitude.unique()) == len(longitudes)-1)
# assert len(final_df) == len(test_df) + len(train_df)
# train_df.head()

In [12]:
# X_train = train_df.iloc[:, 0:-1]
# y_train = train_df.iloc[:, -1]
# X_train, y_train = np.array(X_train), np.array(y_train)#.reshape(-1, 1)

# X_test = test_df.iloc[:, 0:-1]
# y_test = test_df.iloc[:, -1]
# X_test, y_test = np.array(X_test), np.array(y_test)#.reshape(-1, 1)
# X_train.shape, y_train.shape, X_test.shape, y_test.shape

In [13]:
# X_train[0]

In [14]:
# X_train_scaled = scale_data(X_train)
# X_train_scaled

In [15]:
# model = ffnn(X_train, y_train, 500, 'RMSProp')

In [16]:
# y_pred = model.predict(X_test) 
# y_pred.shape, y_test.reshape(-1, 1).shape

In [17]:
# np.min(y_pred), np.mean(y_pred), np.max(y_pred)

In [18]:
# mean_squared_error(y_test, y_pred)

In [19]:
# y_train_pred = model.predict(X_train)
# mean_squared_error(y_train, y_train_pred)

#### end here

In [20]:
def cross_validation(final_df, idx):
    device_indices = final_df[final_df.latitude==latitudes[idx]].index
    device_df = kampala_df[kampala_df.device_number == device_ids[idx]]
#     assert(len(device_indices) == len(device_df)-device_df.pm2_5_calibrated_value.isna().sum())
    
    test_df = final_df.loc[device_indices]
#     assert(len(test_df.longitude.unique()) == 1)
    
    train_df = pd.concat([final_df, test_df]).drop_duplicates(keep=False)
#     assert(len(train_df.longitude.unique()) == len(longitudes)-1)
#     assert len(final_df) == len(test_df) + len(train_df)
    
    X_train = train_df.iloc[:, 0:-1]
    y_train = train_df.iloc[:, -1]
    X_train, y_train = np.array(X_train), np.array(y_train)#.reshape(-1, 1)
#     X_train_scaled = scale_data(X_train)
    
    X_test = test_df.iloc[:, 0:-1]
    y_test = test_df.iloc[:, -1]
    X_test, y_test = np.array(X_test), np.array(y_test)#.reshape(-1, 1)
#     X_test_scaled = scale_data(X_test)
    
    model = ffnn(X_train, y_train)#, optimizer='Adam', dropout=0.8)
    y_pred = model.predict(X_test) 
    
    rmse = sqrt(mean_squared_error(y_test, y_pred))
    mape = mean_absolute_percentage_error(y_test, y_pred)
    return rmse, mape

In [21]:
latitudes

array([0.35698941, 0.31197891, 0.35068083, 0.3614801 , 0.27305058,
       0.34483917, 0.32907575, 0.3238265 , 0.37307176, 0.3841366 ,
       0.31079195, 0.36384297, 0.37791733, 0.30562933, 0.21854908,
       0.27221229, 0.35192674, 0.29776972, 0.35135358, 0.3024902 ,
       0.34025631, 0.33906876, 0.28461732, 0.33555934, 0.29082243,
       0.35553887, 0.35421934, 0.31772521, 0.06349111, 0.23247482,
       0.28349049, 0.39542619, 0.32005559, 0.29030967, 0.30746479])

In [22]:
latitudes[:10]

array([0.35698941, 0.31197891, 0.35068083, 0.3614801 , 0.27305058,
       0.34483917, 0.32907575, 0.3238265 , 0.37307176, 0.3841366 ])

In [23]:
rmse_list, mape_list = [], []
# for i in range(len(latitudes[30:])):
lat_list = list(latitudes)
for lat in latitudes[30:]:
    i = lat_list.index(lat)
#     print(i)
    try:
        rmse, mape = cross_validation(final_df, i)
        rmse_list.append(rmse)
        mape_list.append(mape)
        print(f'Location {i} successful')
    except Exception as e:
        print(e)
        print(f'Location {i} failed')
#         break

mean_rmse = np.mean(rmse_list)
mean_mape = np.mean(mape_list)
mean_rmse, mean_mape

Epoch 1/1000


2023-07-28 21:33:56.756608: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-07-28 21:33:56.756644: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: PL1207-PRO.paris.inria.fr
2023-07-28 21:33:56.756647: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: PL1207-PRO.paris.inria.fr
2023-07-28 21:33:56.756758: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 525.125.6
2023-07-28 21:33:56.756782: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: NOT_FOUND: could not find kernel module information in driver version file contents: "NVRM version: NVIDIA UNIX Open Kernel Module for x86_64  525.125.06  Release Build  (dvs-builder@U16-A23-14-1)  Tue May 30 05:12:48 UTC 2023
GCC version:  gcc ve

1167/1167 [==============================] - 1s 707us/step - loss: 17111644.0000 - val_loss: 6372633.0000
Epoch 2/1000
1167/1167 [==============================] - 1s 726us/step - loss: 3202254.2500 - val_loss: 635990.3750
Epoch 3/1000
1167/1167 [==============================] - 1s 724us/step - loss: 540654.8750 - val_loss: 195038.9375
Epoch 4/1000
1167/1167 [==============================] - 1s 663us/step - loss: 72455.0859 - val_loss: 1762.8981
Epoch 5/1000
1167/1167 [==============================] - 1s 664us/step - loss: 1569.6456 - val_loss: 1686.1055
Epoch 6/1000
1167/1167 [==============================] - 1s 636us/step - loss: 1496.3103 - val_loss: 1612.0668
Epoch 7/1000
1167/1167 [==============================] - 1s 707us/step - loss: 1426.0131 - val_loss: 1540.7108
Epoch 8/1000
1167/1167 [==============================] - 1s 732us/step - loss: 1358.1097 - val_loss: 1472.0205
Epoch 9/1000
1167/1167 [==============================] - 1s 717us/step - loss: 1292.9436 - val_loss

1167/1167 [==============================] - 1s 699us/step - loss: 530.2562 - val_loss: 623.5681
Epoch 148/1000
1167/1167 [==============================] - 1s 633us/step - loss: 530.2606 - val_loss: 623.5541
Epoch 149/1000
1167/1167 [==============================] - 1s 702us/step - loss: 530.2567 - val_loss: 623.5422
Epoch 150/1000
1167/1167 [==============================] - 1s 741us/step - loss: 530.2573 - val_loss: 623.5428
Epoch 151/1000
1167/1167 [==============================] - 1s 787us/step - loss: 530.2604 - val_loss: 623.5469
Epoch 152/1000
1167/1167 [==============================] - 1s 693us/step - loss: 530.2570 - val_loss: 623.5284
Epoch 153/1000
1167/1167 [==============================] - 1s 686us/step - loss: 530.2592 - val_loss: 623.5444
Epoch 154/1000
1167/1167 [==============================] - 1s 716us/step - loss: 530.2583 - val_loss: 623.5377
Epoch 155/1000
1167/1167 [==============================] - 1s 729us/step - loss: 530.2576 - val_loss: 623.5405
Epoch 1

1167/1167 [==============================] - 1s 687us/step - loss: 530.2571 - val_loss: 623.5491
Epoch 294/1000
1167/1167 [==============================] - 1s 658us/step - loss: 530.2581 - val_loss: 623.5568
Epoch 295/1000
1167/1167 [==============================] - 1s 670us/step - loss: 530.2585 - val_loss: 623.5563
Epoch 296/1000
1167/1167 [==============================] - 1s 782us/step - loss: 530.2575 - val_loss: 623.5598
Epoch 297/1000
1167/1167 [==============================] - 1s 725us/step - loss: 530.2576 - val_loss: 623.5964
Epoch 298/1000
1167/1167 [==============================] - 1s 623us/step - loss: 530.2593 - val_loss: 623.5898
Epoch 299/1000
1167/1167 [==============================] - 1s 720us/step - loss: 530.2595 - val_loss: 623.5698
Epoch 300/1000
1167/1167 [==============================] - 1s 666us/step - loss: 530.2592 - val_loss: 623.5558
Epoch 301/1000
1167/1167 [==============================] - 1s 660us/step - loss: 530.2603 - val_loss: 623.5461
Epoch 3

...

1150/1150 [==============================] - 1s 712us/step - loss: 535.1606 - val_loss: 467.3417
Epoch 506/1000
1150/1150 [==============================] - 1s 713us/step - loss: 535.5950 - val_loss: 443.5511
Epoch 507/1000
1150/1150 [==============================] - 1s 706us/step - loss: 535.5184 - val_loss: 446.1674
Epoch 508/1000
1150/1150 [==============================] - 1s 731us/step - loss: 534.9921 - val_loss: 438.8632
Epoch 509/1000
1150/1150 [==============================] - 1s 738us/step - loss: 535.5737 - val_loss: 455.4321
Epoch 510/1000
1150/1150 [==============================] - 1s 746us/step - loss: 535.7170 - val_loss: 455.2786
Epoch 511/1000
1150/1150 [==============================] - 1s 693us/step - loss: 535.5653 - val_loss: 439.9651
Epoch 512/1000
1150/1150 [==============================] - 1s 770us/step - loss: 535.3876 - val_loss: 457.9210
Epoch 513/1000
1150/1150 [==============================] - 1s 742us/step - loss: 535.2971 - val_loss: 438.9905
Epoch 5

1150/1150 [==============================] - 1s 668us/step - loss: 535.4190 - val_loss: 454.8268
Epoch 652/1000
1150/1150 [==============================] - 1s 750us/step - loss: 535.2858 - val_loss: 517.8248
Epoch 653/1000
1150/1150 [==============================] - 1s 713us/step - loss: 534.9991 - val_loss: 442.1322
Epoch 654/1000
1150/1150 [==============================] - 1s 721us/step - loss: 534.4561 - val_loss: 443.0385
Epoch 655/1000
1150/1150 [==============================] - 1s 745us/step - loss: 535.1746 - val_loss: 447.0857
Epoch 656/1000
1150/1150 [==============================] - 1s 726us/step - loss: 534.8932 - val_loss: 454.2998
Epoch 657/1000
1150/1150 [==============================] - 1s 679us/step - loss: 535.1637 - val_loss: 439.2233
Epoch 658/1000
1150/1150 [==============================] - 1s 737us/step - loss: 535.1816 - val_loss: 449.9876
Epoch 659/1000
1150/1150 [==============================] - 1s 693us/step - loss: 534.8356 - val_loss: 445.5400
Epoch 6

1150/1150 [==============================] - 1s 752us/step - loss: 534.7765 - val_loss: 441.3758
Epoch 798/1000
1150/1150 [==============================] - 1s 654us/step - loss: 533.9835 - val_loss: 469.0560
Epoch 799/1000
1150/1150 [==============================] - 1s 771us/step - loss: 534.8898 - val_loss: 438.8279
Epoch 800/1000
1150/1150 [==============================] - 1s 703us/step - loss: 534.9961 - val_loss: 447.5427
Epoch 801/1000
1150/1150 [==============================] - 1s 730us/step - loss: 534.7466 - val_loss: 452.9276
Epoch 802/1000
1150/1150 [==============================] - 1s 761us/step - loss: 534.3838 - val_loss: 438.8694
Epoch 803/1000
1150/1150 [==============================] - 1s 725us/step - loss: 534.0374 - val_loss: 454.7801
Epoch 804/1000
1150/1150 [==============================] - 1s 748us/step - loss: 534.7692 - val_loss: 441.6607
Epoch 805/1000
1150/1150 [==============================] - 1s 770us/step - loss: 534.8578 - val_loss: 444.3351
Epoch 8

1150/1150 [==============================] - 1s 698us/step - loss: 534.1835 - val_loss: 444.7044
Epoch 944/1000
1150/1150 [==============================] - 1s 739us/step - loss: 533.7615 - val_loss: 438.8393
Epoch 945/1000
1150/1150 [==============================] - 1s 753us/step - loss: 534.5537 - val_loss: 439.5839
Epoch 946/1000
1150/1150 [==============================] - 1s 726us/step - loss: 534.3325 - val_loss: 441.2274
Epoch 947/1000
1150/1150 [==============================] - 1s 719us/step - loss: 534.2263 - val_loss: 439.5429
Epoch 948/1000
1150/1150 [==============================] - 1s 704us/step - loss: 534.7724 - val_loss: 439.0072
Epoch 949/1000
1150/1150 [==============================] - 1s 709us/step - loss: 534.4285 - val_loss: 460.8539
Epoch 950/1000
1150/1150 [==============================] - 1s 720us/step - loss: 534.0060 - val_loss: 449.4773
Epoch 951/1000
1150/1150 [==============================] - 1s 769us/step - loss: 534.0931 - val_loss: 441.9519
Epoch 9

(25.486878729086257, 0.5635499936803134)

In [27]:
rmse_list = [16.65009880169072,
 19.98036995863874,
 13.91764028126171,
 17.484046821178662,
 26.68229436187206,
 29.440037308275517,
 20.764832241598498,
 17.691179365921325,
 41.53319507274803,
 24.00624406841042, 
 26.388810328618654,
 17.6992044259869,
 22.185177616301566,
 45.72909401409058,
 18.897752769515797,
 23.570531278412595,
 26.122001031985228,
 19.928827439075945,
 17.792197830486618,
 16.83332629375624,
 36.270800154530626,
 17.929597108448,
 17.51318371273514,
 16.367637482495496,
 17.49710580225471,
 21.981710353216318,
 29.918318553418253,
 23.28441545358778,
 18.324540869301387,
 17.62073268648032,
 18.381039725374958, 
 30.071146755152274, 
 18.35583001598065, 
 35.13949841983713
]
np.mean(rmse_list)

22.998600541254085

In [28]:
mape_list = [0.595685697049459,
 1.148333120497191,
 0.795857599406947,
 0.8593620127029122,
 0.450101263422094,
 0.4933467917315183,
 0.6890228191163319,
 0.6878462720820999,
 0.44993798196519147,
 0.4471827068532165,
 0.5931530628085236,
 1.0267007146987739,
 0.7658773303405604,
 0.4664958898318561,
 1.4480858100972571,
 0.5426506646754397,
 0.4289086672435245,
 0.5425005983654357,
 0.8045697347060672,
 0.9270162974175066,
 0.424622448001312,
 0.9719420798263014,
 0.8802535672363451,
 0.7390971050365004,
 0.7909998934513071,
 0.4422157547663432,
 0.4927301155599275,
 0.41504840140878996,
 0.7907503561165418,
 1.0782583725845007,
 0.6931366772511695,
 0.4547678139239904,
 0.6042062347245889,
 0.5020892488215049
]
np.mean(mape_list)

0.6894927383447362